In [1]:
from divisiveclustering.datautils import DataUtils
from divisiveclustering.coresetsUtils import Coreset, normalize_np ,gen_coreset_graph
from divisiveclustering.vqe_utils import kernel_two_local, create_Hamiltonian_for_K2, get_Hamil_variables
from divisiveclustering.quantumutils import get_probs_table
import cudaq
import numpy as np
import warnings
from divisiveclustering.coresetsUtils import gen_coreset_graph
warnings.filterwarnings("ignore")
import pandas as pd
from divisiveclustering.helpers import add_children_to_hc

In [2]:
number_of_qubits = 5
layer_count = 1
parameter_count = 4 * layer_count * (number_of_qubits - 1)

In [3]:
data_utils = DataUtils('../../data')

try:
    raw_data = data_utils.load_dataset()
except:
    raw_data = data_utils.create_dataset(n_samples = 1000)

Data loaded from ../../data/dataset.pickle


In [4]:
coreset_vectors = data_utils.load_dataset("5_coreset_vectors.npy")
coreset_weights = data_utils.load_dataset("5_coreset_weights.npy") 

In [5]:
coreset_points, G, H, weight_matrix, weights = gen_coreset_graph(
    coreset_vectors, coreset_weights, metric="dot"
)

In [6]:
hamiltonian = create_Hamiltonian_for_K2(G, number_of_qubits, weights)

In [11]:
qubits = number_of_qubits
max_iterations = 100
max_shots = 10
parameter_count = 4 * layer_count * (qubits - 1)

optimizer = cudaq.optimizers.COBYLA()
optimizer.initial_parameters = np.random.uniform(-np.pi / 8.0, np.pi / 8.0,
                                         parameter_count)
optimizer.max_iterations = max_iterations

optimal_expectation, optimal_parameters = cudaq.vqe(
    kernel=kernel_two_local(qubits, layer_count),
    spin_operator=hamiltonian,
    optimizer=optimizer,
    parameter_count=parameter_count,
    shots = max_shots)

counts = cudaq.sample(kernel_two_local(qubits, layer_count), optimal_parameters, shots_count = max_shots)

In [12]:
print(counts)

{ 11001:9 00110:1 }

